# Amazon SageMaker Batch Transform

## Background
This purpose of this notebook is to train a model using SageMaker's XGBoost and UCI's breast cancer diagnostic data set to illustrate at how to run batch inferences and how to use the Batch Transform I/O join feature. UCI's breast cancer diagnostic data set is available at https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29. The data set is also available on Kaggle at https://www.kaggle.com/uciml/breast-cancer-wisconsin-data. The purpose here is to use this data set to build a predictve model of whether a breast mass image indicates benign or malignant tumor. 



---

## Setup

Let's start by specifying:

* The SageMaker role arn used to give training and batch transform access to your data. The snippet below will use the same role used by your SageMaker notebook instance. Otherwise, specify the full ARN of a role with the SageMakerFullAccess policy attached.
* The S3 bucket that you want to use for training and storing model objects.

In [ ]:
!python -m pip install --upgrade pip --quiet
!pip install -U awscli --quiet
!pip install sagemaker --upgrade

In [ ]:
import os
import boto3
import sagemaker
from time import gmtime, strftime
from datetime import datetime

boto_session = boto3.session.Session()
sm_session = sagemaker.session.Session()
sm_role = sagemaker.get_execution_role()
region = boto_session.region_name
s3_bucket = sm_session.default_bucket()
bucket_prefix = "DEMO-breast-cancer-prediction-xgboost-highlevel"
resource_name = "BatchInferenceDemo-{}-{}"

print(f"Will use bucket '{s3_bucket}' for storing all resources related to this notebook")
print(f"Using Role: {sm_role}")

---
## Data sources

> Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

> Breast Cancer Wisconsin (Diagnostic) Data Set [https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)].

> _Also see:_ Breast Cancer Wisconsin (Diagnostic) Data Set [https://www.kaggle.com/uciml/breast-cancer-wisconsin-data].

## Data preparation


Let's download the data and save it in the local folder with the name data.csv and take a look at it.

In [ ]:
import pandas as pd
import numpy as np

s3 = boto3.client("s3")

filename = "wdbc.csv"
s3.download_file("sagemaker-sample-files", "datasets/tabular/breast_cancer/wdbc.csv", filename)
data = pd.read_csv(filename, header=None)

# specify columns extracted from wbdc.names
data.columns = [
    "id",
    "diagnosis",
    "radius_mean",
    "texture_mean",
    "perimeter_mean",
    "area_mean",
    "smoothness_mean",
    "compactness_mean",
    "concavity_mean",
    "concave points_mean",
    "symmetry_mean",
    "fractal_dimension_mean",
    "radius_se",
    "texture_se",
    "perimeter_se",
    "area_se",
    "smoothness_se",
    "compactness_se",
    "concavity_se",
    "concave points_se",
    "symmetry_se",
    "fractal_dimension_se",
    "radius_worst",
    "texture_worst",
    "perimeter_worst",
    "area_worst",
    "smoothness_worst",
    "compactness_worst",
    "concavity_worst",
    "concave points_worst",
    "symmetry_worst",
    "fractal_dimension_worst",
]

# save the data
data.to_csv("data.csv", sep=",", index=False)

data.sample(8)

#### Key observations:
* The data has 569 observations and 32 columns.
* The first field is the 'id' attribute that we will want to drop before batch inference and add to the final inference output next to the probability of malignancy.
* Second field, 'diagnosis', is an indicator of the actual diagnosis ('M' = Malignant; 'B' = Benign).
* There are 30 other numeric features that we will use for training and inferencing.

Let's replace the M/B diagnosis with a 1/0 boolean value. 

In [ ]:
data["diagnosis"] = data["diagnosis"].apply(lambda x: ((x == "M")) + 0)
data.sample(8)

Let's split the data and set 10% aside for our batch inference job. In addition, let's drop the 'id' field on the training set and validation set as 'id' is not a training feature. For our batch set however, we keep the 'id' feature. We'll want to filter it out prior to running our inferences so that the input data features match the ones of training set and then ultimately, we'll want to join it with inference result. We are however dropping the diagnosis attribute for the batch set since this is what we'll try to predict.

Let's upload those data sets in S3

In [ ]:
rand_split = np.random.rand(len(data))
batch_list = rand_split >= 0.9
data_batch = data[batch_list].drop(["diagnosis"], axis=1)
data_batch_noID = data_batch.drop(["id"], axis=1)

batch_file = "batch_data.csv"
data_batch.to_csv(batch_file, index=False, header=False)
sm_session.upload_data(batch_file, key_prefix="{}/batch".format(bucket_prefix))

batch_file_noID = "batch_data_noID.csv"
data_batch_noID.to_csv(batch_file_noID, index=False, header=False)
sm_session.upload_data(batch_file_noID, key_prefix="{}/batch".format(bucket_prefix))

## Create a SageMaker Model

Specify the location of the pre-trained model stored in Amazon S3. This example uses a pre-trained XGBoost model name demo-xgboost-model.tar.gz. The full Amazon S3 URI is stored in a string variable model_url.

In [ ]:
model_s3_key = f"{bucket_prefix}/model.tar.gz"
model_url = f"s3://{s3_bucket}/{model_s3_key}"
print(f"Uploading Model to {model_url}")

with open("model/model.tar.gz", "rb") as model_file:
    boto_session.resource("s3").Bucket(s3_bucket).Object(model_s3_key).upload_fileobj(model_file)

Specify a primary container. For the primary container, you specify the Docker image that contains inference code, artifacts (from prior training), and a custom environment map that the inference code uses when you deploy the model for predictions. In this example, we specify an XGBoost built-in algorithm container image.

In [ ]:
from sagemaker import image_uris

# Specify an AWS container image and region as desired
container = image_uris.retrieve(region=region, framework="xgboost", version="0.90-1")

Create a SageMaker Model by specifying the name, the role (the ARN of the IAM role that Amazon SageMaker can assume to access model artifacts/ docker images for deployment), and the image_uri of the XGBoost built-in algorithm container image.

In [ ]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

model_name = resource_name.format("Model", datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))

model_predictor = Model(
    name=model_name,
    image_uri=container,
    model_data=model_url,
    role=sm_role,
    predictor_cls=Predictor,
)
model_name

In [ ]:
# Deploy the model
instance_count=1,
instance_type="ml.m5.4xlarge"

predictor = model_predictor.deploy(
        instance_type='ml.m5.4xlarge',
        initial_instance_count=1)


---

## Batch Transform

In SageMaker Batch Transform, we introduced 3 new attributes - __input_filter__, __join_source__ and __output_filter__. In the below cell, we use the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick-off several Batch Transform jobs using different configurations of these 3 new attributes. Please refer to [this page](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html) to learn more about how to use them.




#### 1. Create a transform job with the default configurations
Let's first skip these 3 new attributes and inspect the inference results. We'll use it as a baseline to compare to the results with data processing.

In [ ]:
# %%time

from sagemaker.transformer import Transformer

sm_transformer = Transformer(model_name=model_name,
                          instance_count=1,
                          instance_type='ml.m4.xlarge')

#Start the Batch Transform job

input_location = "s3://{}/{}/batch/{}".format(
    s3_bucket, bucket_prefix, batch_file_noID
)  # use input data without ID column

sm_transformer.transform(input_location, content_type="text/csv", split_type="Line")


Let's inspect the output of the Batch Transform job in S3. It should show the list probabilities of tumors being malignant.

In [ ]:
import re


def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)
    s3.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=None)

Let's inspect the output of the Batch Transform job in S3. It should show the list of tumors identified by their original feature columns and their corresponding probabilities of being malignant.

In [ ]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file_noID)
output_df.head(8)